In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import geopandas as gpd
import random
from scipy.stats import truncnorm, norm
from collections import defaultdict
import math
from shapely.ops import transform
from pyproj import Geod
import pyarrow
from datetime import datetime
import sys
import importlib
from dataclasses import dataclass
import yaml
from typing import Union

In [4]:
# добавляем папку modules в sys.path, чтобы импортировать функции созданные ранее

# sys.path.append(os.path.abspath(os.path.join("..", "modules")))

In [501]:
# Функции генерации паттернов времени и базовые функции времени созданные ранее
import data_generator.fraud.time
import data_generator.fraud.txndata
import data_generator.configs
importlib.reload(data_generator.fraud.time)
importlib.reload(data_generator.fraud.txndata)
importlib.reload(data_generator.configs)
from data_generator.general_time import *
from data_generator.utils import build_transaction
from data_generator.fraud.txndata import FraudTxnPartData, TransAmount, sample_category
from data_generator.fraud.time import derive_from_last_time
from data_generator.configs import PurchBehaviorHandler

In [2]:
np.set_printoptions(suppress=True)
pd.set_option('display.max_columns', None)

In [3]:
os.getcwd()

'C:\\Users\\iaros\\My_documents\\Education\\projects\\fraud_detection_01\\notebooks'

In [4]:
os.chdir("..")

In [5]:
os.getcwd()

'C:\\Users\\iaros\\My_documents\\Education\\projects\\fraud_detection_01'

## Загрузка конфигураций

In [6]:
# Общие настройки
with open("./config/base.yaml") as f:
    base_cfg = yaml.safe_load(f)
# Настройки фрода
with open("./config/fraud.yaml") as f:
    fraud_cfg = yaml.safe_load(f)
# Настройки фрода для дропов
with open("./config/drops.yaml", encoding="utf8") as f:
    drops_cfg = yaml.safe_load(f)
# Настройки времени
with open("./config/time.yaml") as f:
    time_cfg = yaml.safe_load(f)

# Доп. данные для генерации фрод транзакций

**Загрузка данных:**
- оффлайн мерчантов
- онлайн мерчантов
- девайсов клиентов и мошенников
- городов с полигонами
- городов с координатами центров
- клиентов
- фрод IP адресов
- категорий и их характеристик
- категорий для дропов покупателей
- Счетов клиентов
- Внешних счетов

In [612]:
offline_merchants = gpd.read_file("./data/cleaned_data/offline_merchants_points.gpkg")
online_merchant_ids = pd.read_csv("./data/cleaned_data/online_merchant_ids.csv").iloc[:, 0] # нужны в виде серии
client_devices = pd.read_csv("./data/cleaned_data/client_devices.csv")
fraud_devices = pd.read_csv("./data/cleaned_data/fraud_devices.csv")
districts_ru = gpd.read_file("./data/cleaned_data/district_ru.gpkg")
area_centers = gpd.read_file("./data/cleaned_data/area_centers.gpkg")
clients_with_geo = gpd.read_file("./data/cleaned_data/clients_with_geo.gpkg") 
fraud_ips = gpd.read_file("./data/cleaned_data/fraud_ips.gpkg")
cat_stats_full = pd.read_csv("./data/cleaned_data/cat_stats_full.csv")
cat_fraud_amts = pd.read_csv("./data/cleaned_data/cat_fraud_amts.csv")
drop_purch_cats = pd.read_csv("./data/cleaned_data/drop_purch_cats.csv")
accounts = pd.read_csv("./data/generated_data/accounts.csv")
outer_accounts = pd.read_csv("./data/generated_data/outer_accounts.csv").iloc[:, 0] # нужны в виде серии

**Функция получения случайных значений из `truncnorm` распределения**
- пойдет в модуль `utils`

In [14]:
def get_values_from_truncnorm(low_bound, high_bound, mean, std, size=1):
    """
    Сгенерировать массив чисел из обрезанного нормального распределения.
    Можно сгенерировать массив с одним числом
    ------------
    low_bound - float, int. Нижняя граница значений
    high_bound - float, int. Верхняя граница значений 
    mean - float, int. Среднее
    std - float, int. Стандартное отклонение
    size - Количество чисел в возвращаемом массиве
    ------------
    Возвращает np.ndarray
    """
    return truncnorm.rvs((low_bound - mean) / std, (high_bound - mean) / std, loc=mean, scale=std, size=size)

## Класс `DropDistributorCfg` конфиги для дропов распределителей
- конфигурации на основании которых будут генерироваться транзакции дропов распределителей,
  которые заняты распределением полученных денег: переводами, снятиями, покупкой криптовалюты.

In [52]:
# ----------- ВСТАВИТЬ ГОТОВЫЙ КЛАСС СЮДА -------------------------

## Класс `DropPurchaserCfg` конфиги для дропов покупателей
- конфигурации на основании которых будут генерироваться транзакции дропов покупателей,
  которые заняты покупкой товаров на полученные деньги.

In [ ]:
# ----------- ВСТАВИТЬ ГОТОВЫЙ КЛАСС СЮДА -------------------------

## Класс `DropConfigBuilder`
- создает объект `DropDistributorCfg` или `DropPurchaserCfg`

In [1]:
# ----------- ВСТАВИТЬ ГОТОВЫЙ КЛАСС СЮДА -------------------------

### **Тест `DropConfigBuilder`**

In [1]:
# Временный импорт
import os
import yaml
import pandas as pd
import numpy as np
import geopandas as gpd
os.chdir("..")

# Общие настройки
with open("./config/base.yaml") as f:
    base_cfg = yaml.safe_load(f)
# Настройки фрода
with open("./config/fraud.yaml") as f:
    fraud_cfg = yaml.safe_load(f)
# Настройки фрода для дропов
with open("./config/drops.yaml", encoding="utf8") as f:
    drops_cfg = yaml.safe_load(f)
# Настройки времени
with open("./config/time.yaml") as f:
    time_cfg = yaml.safe_load(f)

clients_with_geo = gpd.read_file("./data/cleaned_data/clients_with_geo.gpkg") 

In [2]:
from data_generator.fraud.drops.build.config import DropConfigBuilder

In [3]:
drop_cfg_build = DropConfigBuilder(base_cfg=base_cfg, fraud_cfg=fraud_cfg, drop_cfg=drops_cfg)

In [4]:
clients_sample = gpd.read_file("./data/cleaned_data/clients_sample.gpkg")

In [5]:
# Клиенты незадействованные ранее в легальных транзакциях и compromised фроде

not_used_clients = clients_with_geo.loc[~clients_with_geo.client_id.isin(clients_sample.client_id)].copy()

**Тест `get_clients_for_drops`**

**генерируем `purchaser`-ов**  
Файл с `distributor`-ами существует

In [6]:
dist_count = drop_cfg_build.estimate_drops_count(drop_type="distributor")
dist_count

28

In [7]:
dist_drops = not_used_clients.sample(n=dist_count).reset_index(drop=True)

In [8]:
dist_drops.to_file("./data/generated_data/dist_drops.gpkg", layer='layer_name', driver="GPKG")

In [9]:
drop_cfg_build.get_clients_for_drops(drop_type="purchaser")
print(drop_cfg_build.drops.shape[0])
drop_cfg_build.drops.head()

28


,client_id,district_id,birth_date,sex,region,area,timezone,lat,lon,population,home_ip,geometry
0,2380,40,1927-08-17,male,Пензенская,Пенза,UTC+3,53.175388,45.034741,519592,2.60.8.206,"MULTIPOLYGON (((44.82976 53.24571, 44.83161 53..."
1,696,28,1952-12-01,female,Челябинская,Магнитогорск,UTC+5,53.407189,58.979143,408401,2.60.2.153,"MULTIPOLYGON (((58.89722 53.47195, 58.89909 53..."
2,156,54,1930-07-22,female,Свердловская,Екатеринбург,UTC+5,56.838633,60.605489,1377738,2.60.0.147,"MULTIPOLYGON (((60.00708 56.80588, 60.01103 56..."
3,740,55,1923-11-07,male,Самарская,Тольятти,UTC+4,53.520644,49.389461,719484,2.60.2.196,"MULTIPOLYGON (((49.14714 53.55389, 49.15065 53..."
4,3632,15,1943-09-29,female,Ульяновская,Ульяновск,UTC+4,54.308067,48.374872,613793,2.60.13.113,"MULTIPOLYGON (((48.25014 54.29555, 48.26023 54..."


In [10]:
purch_drops = drop_cfg_build.drops

In [11]:
samp_and_dist = pd.concat([clients_sample, dist_drops], ignore_index=True)
assert samp_and_dist.loc[samp_and_dist.client_id.isin(purch_drops.client_id)].empty

**генерируем `purchaser`-ов**  
Файл с `distributor`-ами НЕ существует

In [12]:
path1 = "./data/generated_data/dist_drops.gpkg"
if os.path.exists(path1):
    os.remove(path1)
drop_cfg_build.get_clients_for_drops(drop_type="purchaser")
print(drop_cfg_build.drops.shape[0])
drop_cfg_build.drops.head()

28


,client_id,district_id,birth_date,sex,region,area,timezone,lat,lon,population,home_ip,geometry
0,144,65,1958-12-06,male,Ростовская,Таганрог,UTC+3,47.209491,38.935154,257692,2.60.0.135,"MULTIPOLYGON (((38.79298 47.23052, 38.7937 47...."
1,3717,71,1925-01-30,male,Мурманская,Мурманск,UTC+3,69.007696,33.068602,307664,2.60.13.193,"MULTIPOLYGON (((32.93287 68.90211, 32.93588 68..."
2,2141,28,1951-09-06,male,Челябинская,Магнитогорск,UTC+5,53.407189,58.979143,408401,2.60.7.233,"MULTIPOLYGON (((58.89722 53.47195, 58.89909 53..."
3,8217,69,1978-08-02,male,Иркутская,Иркутск,UTC+8,52.286351,104.280655,587225,2.60.19.84,"MULTIPOLYGON (((104.05892 52.39701, 104.05964 ..."
4,6114,66,1940-02-15,female,Воронежская,Воронеж,UTC+3,51.659238,39.196828,889680,2.60.18.184,"MULTIPOLYGON (((39.01323 51.60446, 39.01766 51..."


In [13]:
purch_drops2 = drop_cfg_build.drops

In [14]:
assert clients_sample.loc[clients_sample.client_id.isin(purch_drops2.client_id)].empty

**генерируем `distributor-ов`**  
Файл с `purchaser`-ами существует

In [15]:
purch_count = drop_cfg_build.estimate_drops_count(drop_type="purchaser")
purch_drops = not_used_clients.sample(n=purch_count).reset_index(drop=True)
dist_drops.to_file("./data/generated_data/dist_drops.gpkg", layer='layer_name', driver="GPKG")

In [16]:
path2 = "./data/generated_data/purchase_drops.gpkg"
assert os.path.exists(path2), "File does not exist"

drop_cfg_build.get_clients_for_drops(drop_type="distributor")
print(drop_cfg_build.drops.shape[0])
drop_cfg_build.drops.head()

28


,client_id,district_id,birth_date,sex,region,area,timezone,lat,lon,population,home_ip,geometry
0,625,60,1978-11-24,female,Хабаровский,Хабаровск,UTC+10,48.464799,135.059881,577668,2.60.2.84,"MULTIPOLYGON (((134.89789 48.4426, 134.89823 4..."
1,4434,64,1964-03-29,male,Нижегородская,Нижний Новгород,UTC+3,56.324209,44.005395,1250615,2.60.16.97,"MULTIPOLYGON (((43.72068 56.21979, 43.7211 56...."
2,1435,52,1953-06-25,male,Челябинская,Челябинск,UTC+5,55.160366,61.400786,1130273,2.60.5.80,"MULTIPOLYGON (((61.1436 55.05351, 61.1436 55.0..."
3,343,44,1919-11-19,male,Ставропольский,Ставрополь,UTC+3,45.044544,41.969017,398266,2.60.1.68,"MULTIPOLYGON (((41.81047 45.00561, 41.81108 45..."
4,9450,68,1943-10-11,female,Самарская,Самара,UTC+4,53.195166,50.106769,1164900,2.60.19.150,"MULTIPOLYGON (((49.73139 53.48063, 49.73797 53..."


In [17]:
dist_drops1 = drop_cfg_build.drops

In [18]:
samp_and_purch = pd.concat([clients_sample, purch_drops], ignore_index=True)
assert samp_and_purch.loc[samp_and_purch.client_id.isin(dist_drops1.client_id)].empty

**генерируем `distributor-ов`**  
Файл с `purchaser`-ами НЕ существует

In [19]:
path2 = "./data/generated_data/purchase_drops.gpkg"
if os.path.exists(path2):
    os.remove(path2)
drop_cfg_build.get_clients_for_drops(drop_type="distributor")
print(drop_cfg_build.drops.shape[0])
drop_cfg_build.drops.head()

28


,client_id,district_id,birth_date,sex,region,area,timezone,lat,lon,population,home_ip,geometry
0,3126,47,1938-01-13,male,Алтайский,Барнаул,UTC+7,53.348115,83.779836,635585,2.60.11.146,"MULTIPOLYGON (((83.5329 53.34391, 83.53472 53...."
1,283,73,1933-11-04,male,Оренбургская,Оренбург,UTC+5,51.787519,55.101738,570329,2.60.1.11,"MULTIPOLYGON (((55.01926 51.78958, 55.01948 51..."
2,5699,73,1950-10-08,male,Оренбургская,Оренбург,UTC+5,51.787519,55.101738,570329,2.60.18.148,"MULTIPOLYGON (((55.01926 51.78958, 55.01948 51..."
3,258,22,1937-05-31,male,Забайкальский,Чита,UTC+9,52.034013,113.499488,323964,2.60.0.243,"MULTIPOLYGON (((113.13361 51.99927, 113.13574 ..."
4,831,2,1948-05-02,male,Мордовия,Саранск,UTC+3,54.180760,45.186226,297425,2.60.3.21,"MULTIPOLYGON (((45.05548 54.18571, 45.06143 54..."


In [20]:
dist_drops2 = drop_cfg_build.drops

In [21]:
assert clients_sample.loc[clients_sample.client_id.isin(dist_drops2.client_id)].empty

**Тест `build_dist_cfg`**  
**создание объекта `DropDistributorCfg`**

In [1159]:
dist_configs = drop_cfg_build.build_dist_cfg()

Конфиги загружаются временно тут

In [1067]:
# Общие настройки
with open("./config/base.yaml") as f:
    base_cfg = yaml.safe_load(f)
# Настройки фрода
with open("./config/fraud.yaml") as f:
    fraud_cfg = yaml.safe_load(f)
# Настройки фрода для дропов
with open("./config/drops.yaml", encoding="utf8") as f:
    drops_cfg = yaml.safe_load(f)
# Настройки времени
with open("./config/time.yaml") as f:
    time_cfg = yaml.safe_load(f)

## Объект датакласса `DropDistributorCfg` конфиги для дропов распределителей

In [1015]:
# Импорт на время разработки
import data_generator.configs
importlib.reload(data_generator.configs)
from data_generator.configs import DropDistributorCfg

**Доп. конфиги из `drop.yaml`**

In [1040]:
# временно

# Общие настройки
with open("./config/base.yaml") as f:
    base_cfg = yaml.safe_load(f)
# Настройки фрода
with open("./config/fraud.yaml") as f:
    fraud_cfg = yaml.safe_load(f)
# Настройки фрода для дропов
with open("./config/drops.yaml", encoding="utf8") as f:
    drops_cfg = yaml.safe_load(f)
# Настройки времени
with open("./config/time.yaml") as f:
    time_cfg = yaml.safe_load(f)

In [1042]:
dist_configs = DropDistributorCfg(clients=dist_drop_clients, timestamps=timestamps, accounts=accounts, \
                                  outer_accounts=outer_accounts, client_devices=client_devices, \
                                  online_merchant_ids=online_merchant_ids, cities=districts_ru, in_lim=dist_in_lim, 
                                  out_lim=dist_out_im, period_in_lim=period_in_lim, period_out_lim=period_out_lim, \
                                  lag_interval=dist_lag_int, two_way_delta=two_way_delta, pos_delta=pos_delta, \
                                  split_rate=split_rate, chunks=chunks, inbound_amt=dist_inbound, round=dist_round, \
                                  trf_max=dist_trf_max, reduce_share=dist_reduce, attempts=dist_att, to_drops=to_drops, \
                                  crypto_rate=crypto_rate
                                 )

## Класс `DropAccountHandler`

In [484]:
class DropAccountHandler:
    """
    Генератор номеров счетов входящих/исходящих транзакций.
    Учет использованных счетов.
    """
    def __init__(self, configs: DropConfigs, client_id, used_accounts=pd.Series(name="account_id")):
        """
        configs - pd.DataFrame. Данные для создания транзакций: номера счетов клиентов и внешних счетов, и др.
        client_id - int. id текущего дропа.
        used_accounts - pd.Series. Счета на которые дропы уже отправляли деньги.
        """
        self.accounts = configs.accounts
        self.outer_accounts = configs.outer_accounts
        self.client_id = client_id
        # self.account = 0
        self.account = configs.accounts.loc[configs.accounts.client_id == client_id, "account_id"].iat[0]
        self.used_accounts = used_accounts

    def get_account(self, to_drop):
        """
        Номер счета входящего/исходящего перевода
        to_drop - bool. Перевод другому дропу в нашем банке или нет.
        """
        # Если отправляем/получаем из другого банка.  
        if not to_drop:
            # Семплируем номер внешнего счета который еще не использовался
            account = self.outer_accounts.loc[~(self.outer_accounts.isin(self.used_accounts))].sample(1).iat[0]
            # Добавляем этот счет в использованные как последнюю запись в серии
            self.used_accounts.loc[self.used_accounts.shape[0]] = account
            return account
        
        # Если надо отправить другому дропу в нашем банке. При условии что есть другие дропы на текущий момент
        # Фильтруем accounts исключая свой счет и использованные счета, и выбирая дропов. Для случая если to_drop
        drop_accounts = self.accounts.loc[(self.accounts.client_id != self.client_id) & (self.accounts.is_drop == True) \
                                          & ~(self.accounts.account_id.isin(self.used_accounts))]
        
        # Если счетов дропов ещё нет. Берем внешний неиспользованный счет
        if drop_accounts.empty:
            account = self.outer_accounts.loc[~(self.outer_accounts.isin(self.used_accounts))].sample(1).iat[0]
            self.used_accounts.loc[self.used_accounts.shape[0]] = account
            return account

        # Дропы есть
        account = drop_accounts.account_id.sample(1).iat[0]
        # Добавляем этот счет в использованные как последнюю запись в серии
        self.used_accounts.loc[self.used_accounts.shape[0]] = account
        return account


    def reset_cache(self):
        """
        Сброс кэшированных значений.
        Это только использованные номера счетов
        """
        self.used_accounts = pd.Series(name="account_id")

**Тест `DropAccountHandler`**

In [1]:
# Временный импорт
import os
import yaml

os.chdir("..")

# Общие настройки
with open("./config/base.yaml") as f:
    base_cfg = yaml.safe_load(f)
# Настройки фрода
with open("./config/fraud.yaml") as f:
    fraud_cfg = yaml.safe_load(f)
# Настройки фрода для дропов
with open("./config/drops.yaml", encoding="utf8") as f:
    drops_cfg = yaml.safe_load(f)
# Настройки времени
with open("./config/time.yaml") as f:
    time_cfg = yaml.safe_load(f)

In [12]:
# импорт на время разработки

from data_generator.fraud.drops.build.config import DropConfigBuilder
from data_generator.fraud.drops.base import DropAccountHandler

drop_cfg_build = DropConfigBuilder(base_cfg=base_cfg, fraud_cfg=fraud_cfg, drop_cfg=drops_cfg)
dist_configs = drop_cfg_build.build_dist_cfg()
# purch_configs = drop_cfg_build.build_purch_cfg()

In [14]:
drop_acc_hand1 = DropAccountHandler(configs=dist_configs)
# drop_acc_hand1 = DropAccountHandler(configs=purch_configs)

In [15]:
drop_acc_hand1.client_id = 1

In [16]:
drop_acc_hand1.get_account(own=True)
drop_acc_hand1.account

np.int64(10000)

In [17]:
drop_acc_hand1.get_account()

np.int64(17191)

In [7]:
drop_acc_hand1.used_accounts

0    21539
Name: account_id, dtype: int64

In [16]:
drop_acc_hand1.label_drop()

In [17]:
drop_acc_hand1.accounts.head(2)

,client_id,account_id,is_drop
0,1,10000,True
1,2,10001,False


In [1274]:
drop_acc_hand1.client_id = 99

In [1275]:
drop_acc_hand1.get_account(own=True)
drop_acc_hand1.label_drop()

In [1276]:
drop_acc_hand1.accounts.query("client_id == 99")

,client_id,account_id,is_drop
93,99,10093,True


**`to_drop=True`**  нет дропов в self.accounts  (кроме текущего дропа)

In [1277]:
own_id = drop_acc_hand1.client_id

In [1283]:
drop_acc_hand1.accounts.query("client_id != @own_id and is_drop == True")

,client_id,account_id,is_drop


In [18]:
drop_acc_hand1.reset_cache()
outer_acc1 = drop_acc_hand1.get_account(to_drop=True)
drop_acc_hand1.outer_accounts.loc[drop_acc_hand1.outer_accounts == outer_acc1]

AssertionError: Other functionality works only for DropDistributorCfg object.
            But <class 'data_generator.configs.DropPurchaserCfg'> was passed

In [1182]:
drop_acc_hand1.used_accounts

0    18779
Name: account_id, dtype: int64

**`to_drop=True`** кол-во дропов в self.accounts меньше чем self.min_drops

In [710]:
drop_acc_hand1.reset_cache()
min_drops = dist_configs.to_drops["min_drops"]
print(min_drops)
drop_acc_hand1.accounts["is_drop"] = False
drop_acc_hand1.get_account(own=True)

accs_samp = drop_acc_hand1.accounts.query("client_id != @own_id").client_id.sample(n=min_drops - 1)
drop_acc_hand1.accounts.loc[drop_acc_hand1.accounts.client_id.isin(accs_samp), "is_drop"] = True
drop_acc_hand1.accounts.query("client_id != @own_id and is_drop == True").shape[0]

6


5

In [711]:
outer_acc2 = drop_acc_hand1.get_account(to_drop=True)
outer_acc2

np.int64(21917)

In [712]:
drop_acc_hand1.outer_accounts.loc[drop_acc_hand1.outer_accounts == outer_acc2]

6548    21917
Name: account_id, dtype: int64

In [713]:
drop_acc_hand1.used_accounts

0    21917
Name: account_id, dtype: int64

**`to_drop=True`** кол-во дропов в self.accounts `>=` self.min_drops

In [714]:
drop_acc_hand1.reset_cache()
min_drops = dist_configs.to_drops["min_drops"]
print(min_drops)
drop_acc_hand1.accounts["is_drop"] = False
drop_acc_hand1.get_account(own=True)

accs_samp3 = drop_acc_hand1.accounts.query("client_id != @own_id").client_id.sample(n=min_drops)
drop_acc_hand1.accounts.loc[drop_acc_hand1.accounts.client_id.isin(accs_samp3), "is_drop"] = True
drop_acc_hand1.accounts.query("client_id != @own_id and is_drop == True").shape[0]

6


6

In [715]:
drop_acc1 = drop_acc_hand1.get_account(to_drop=True)
drop_acc1

np.int64(13694)

In [717]:
drop_acc_hand1.accounts.loc[drop_acc_hand1.accounts.account_id == drop_acc1]

,client_id,account_id,is_drop
3694,3904,13694,True


## Класс `DropAmountHandler`

In [ ]:
# class DropAmountHandler: 
#     """
#     Генератор сумм входящих/исходящих транзакций, сумм снятий.
#     Управление балансом текущего дропа.
#     -------------
#     Атрибуты:
#     balance: float, int. Текущий баланс дропа. По умолчанию 0.
#     batch_txns: int. Счетчик транзакций сделанных в рамках распределения полученной партии денег.
#                       По умолчанию 0.
#     chunk_size: int, float. Последний созданный размер части баланса для перевода по частям
#                              По умолчанию 0.
#     chunks: dict. Содержит ключи:
#         atm_min: int. Минимальная сумма для снятий в банкомате.
#         atm_share: float. Доля от баланса, которую дроп снимает в случае снятия в банкомате
#         low: int. Минимальная сумма исходящего перевода частями.
#         high: int. Максимальная сумма исходящего перевода частями.
#         step: int. Шаг возможных сумм. Чем меньше шаг, тем больше вариантов.
#         rand_rate: float. От 0 до 1.
#                    Процент случаев, когда каждый НЕ первый чанк будет случайным и не зависеть от предыдущего.
#                    Но возможны случайные совпадаения с предыдущим размером чанка.
#                    Доля случайных размеров подряд будет:
#                    p(c) - вероятность взять определенный размер (зависит от размера выборки чанков)
#                    p(r) - rand_rate
#                    p(r) - (p(r) * p(c)). Например p(r) = 0.9; и 5 вариантов размеров чанка - p(c) = 0.20
#                    0.9 - (0.9 * 0.2) = 0.72
#                    В около 72% случаев размеры чанков не будут подряд одинаковыми 
#     inbound_amt: dict. Настройки для сумм входящих транзакций. Содержит ключи:
#         low: int
#         high: int
#         mean: int
#         std: int
#     round: int. Округление целой части сумм транзакций. Напр. 500 значит что суммы будут кратны 500 - кончаться на 500 или 000                  
#     """

#     def __init__(self, configs: DropDistributorCfg | DropPurchaserCfg):
#         """
#         configs: DropDistributorCfg | DropPurchaserCfg. Данные на основании, которых генерируются транзакции.
#                  Отсюда берутся: atm_min, atm_share, min, step, rand_rate.
#         """
#         self.balance = 0
#         self.batch_txns = 0
#         self.chunk_size = 0
#         self.chunks = configs.chunks.copy()
#         self.inbound_amt = configs.inbound_amt.copy()
#         self.round = configs.round
#         # self.atm_min = configs.chunks["atm_min"]
#         # self.atm_share = configs.chunks["atm_share"]
#         # self.min = configs.chunks["min"]
#         # self.max = configs.chunks["max"]
#         # self.step = configs.chunks["step"]
#         # self.rand_rate = configs.chunks["rand_rate"]

#     def update_balance(self, amount, receive=False, declined=False):
#         """
#         Увеличить/уменьшить баланс на указанную сумму
#         -------------------
#         amount - float, int.
#         receive - bool. Входящая ли транзакция. Прибавлять сумму или отнимать.
#         declined - bool. Отклонена ли транзакция или одобрена.
#         """
#         # Не обновлять баланс если транзакция отклонена.
#         if declined:
#             return
            
#         # Увеличить баланс   
#         if receive:
#             self.balance += amount
#             return
            
#         # Уменьшить баланс    
#         self.balance -= amount

#     def receive(self, declined):
#         """
#         Генерация суммы входящего перевода
#         --------------------------
#         declined - bool. Отклонена ли транзакция или одобрена
#         """
#         low = self.inbound_amt["low"]
#         high = self.inbound_amt["high"]
#         mean = self.inbound_amt["mean"]
#         std = self.inbound_amt["std"]

#         # Генерация суммы. Округление целой части при необходимости
#         amount = get_values_from_truncnorm(low_bound=low, high_bound=high, mean=mean, std=std)[0] // self.round * self.round
        
#         # Обновляем баланс если транзакция не отклонена
#         self.update_balance(amount=amount, receive=True, declined=declined)
        
#         return amount

#     def get_chunk_size(self, online=False):
#         """
#         Вернуть случайный размер суммы перевода для перевода по частям
#         либо вернуть долю от баланса для снятия/перевода по частям.
#         -------------------------------
#         online - bool. Онлайн или оффлайн. Перевод или банкомат. Если банкомат, то снимается доля self.chunks["atm_share"] от баланса, 
#                  но не меньше self.chunks["atm_min"]
#         --------------------
#         Возвращает np.int64
#         Результат кэшируется в self.chunk_size
#         """
#         # Если это не первая транзакция в серии транзакции для одной полученной дропом суммы
#         # И случайное число больше rand_rate, то просто возвращаем ранее созданный размер чанка
#         rand_rate = self.chunks["rand_rate"]
#         if self.batch_txns != 0 and np.random.uniform(0,1) > rand_rate:
#             return self.chunk_size

#         # Если снятие
#         if not online:
#             atm_min = self.chunks["atm_min"]
#             atm_share = self.chunks["atm_share"]
#             self.chunk_size = max(atm_min, self.balance * atm_share // self.round * self.round)
#             return self.chunk_size
        
#         # Если перевод. 
#         # Берем лимиты под генерацию массива чанков, в зависимости от
#         # полученной дропом суммы
#         small = self.chunks["rcvd_small"]
#         medium = self.chunks["rcvd_medium"] 
#         large = self.chunks["rcvd_large"]
#         step = self.chunks["step"]

#         if self.balance <= small["limit"]:
#             low = min(self.balance, small["min"]) # но не больше суммы на балансе
#             high = min(self.balance, small["max"]) # но не больше суммы на балансе

#         elif self.balance <= medium["limit"]:
#             low = min(self.balance, medium["min"])
#             high = min(self.balance, medium["max"])

#         else:
#             low = min(self.balance, large["min"])
#             high = min(self.balance, large["max"])
   
#         # прибавим шаг к максимуму, чтобы было понятнее передавать аргументы в конфиге 
#         # и не учитывать исключение значения stop в np.arange
#         sampling_array = np.arange(low, high + step, step)
#         # Если чанк больше бал
#         self.chunk_size = np.random.choice(sampling_array)
#         return self.chunk_size

**Тест `DropAmountHandler`**

In [1]:
# Временный импорт
import os
import yaml

os.chdir("..")

# Общие настройки
with open("./config/base.yaml") as f:
    base_cfg = yaml.safe_load(f)
# Настройки фрода
with open("./config/fraud.yaml") as f:
    fraud_cfg = yaml.safe_load(f)
# Настройки фрода для дропов
with open("./config/drops.yaml", encoding="utf8") as f:
    drops_cfg = yaml.safe_load(f)
# Настройки времени
with open("./config/time.yaml") as f:
    time_cfg = yaml.safe_load(f)

In [3]:
# импорт на время разработки

# import data_generator.fraud.drops.base
# import data_generator.indev
# import data_generator.configs

# importlib.reload(data_generator.indev)
# importlib.reload(data_generator.configs)
# importlib.reload(data_generator.fraud.drops.base)
import pandas as pd
import numpy as np
from data_generator.indev import DropConfigBuilder
# from data_generator.configs import DropDistributorCfg
from data_generator.fraud.drops.base import DropAmountHandler

drop_cfg_build = DropConfigBuilder(base_cfg=base_cfg, fraud_cfg=fraud_cfg, drop_cfg=drops_cfg)
configs1 = drop_cfg_build.build_dist_cfg()
# configs1 = drop_cfg_build.build_purch_cfg()
drop_amt1 = DropAmountHandler(configs=configs1)

In [4]:
drop_amt1.chunks

{'atm_min': 10000,
 'atm_share': {'min': 0.3, 'max': 0.7},
 'step': 1000,
 'rand_rate': 0.9,
 'rcvd_small': {'limit': 10000, 'min': 3000, 'max': 10000},
 'rcvd_medium': {'limit': 30000, 'min': 5000, 'max': 25000},
 'rcvd_large': {'min': 10000, 'max': 40000}}

In [4]:
drop_amt1.balance, drop_amt1.batch_txns, drop_amt1.chunk_size

(0, 0, 0)

In [1187]:
# drop_amt1.chunks, drop_amt1.receive, drop_amt1.round

In [1188]:
drop_amt1.reset_cache()
drop_amt1.balance

0

In [6]:
drop_amt1.inbound_amt

{'low': 5000, 'high': 100000, 'mean': 30000, 'std': 20000}

In [5]:
drop_amt1.balance = 50000
drop_amt1.get_chunk_size(online=False)

29200.0

In [4]:
drop_amt1.reset_cache()
received_list = [drop_amt1.receive(declined=True) for _ in range(10000)]
rcvd_amts_df = pd.DataFrame(pd.Series(received_list, name="rcvd_amt"))
print(f"balance: {drop_amt1.balance}")
rcvd_amts_df.rcvd_amt.agg(["min","max", "mean"])

balance: 50000


min      5000.00
max     99900.00
mean    33808.04
Name: rcvd_amt, dtype: float64

In [11]:
drop_amt1.receive(declined=False)

np.float64(46400.0)

In [12]:
drop_amt1.balance

np.float64(46400.0)

In [1194]:
drop_amt1.receive(declined=False)

np.float64(25400.0)

In [1195]:
drop_amt1.balance

np.float64(89400.0)

In [13]:
print("before: ", drop_amt1.balance)
drop_amt1.update_balance(amount=7000, receive=False, declined=True)
drop_amt1.balance

before:  46400.0


np.float64(46400.0)

**Тест `chunk_size`**

In [689]:
# Значения атрибутов отличные от прописанных в конфиге - закомментить/раскомментить при надобности
# drop_amt1.chunks["low"] = 10000
# drop_amt1.chunks["high"] = 45000
# drop_amt1.chunks["step"] = 1000
# drop_amt1.chunks["atm_min"] = 15000
# drop_amt1.chunks["atm_share"] = 0.4
# drop_amt1.chunks["rand_rate"] = 0.5

In [6]:
drop_amt1.balance = 10000
drop_amt1.batch_txns = 1
drop_amt1.get_chunk_size(online=False), drop_amt1.chunk_size

(10000, 10000)

In [16]:
drop_amt1.reset_cache(life_end=True)
drop_amt1.batch_txns = 1
# drop_amt1.chunk_size = 20000
drop_amt1.balance = drop_amt1.chunks["rcvd_medium"]["limit"] + 30000
chunks_list = [drop_amt1.get_chunk_size(online=False) for _ in range(1000)]
chunks_df = pd.DataFrame(pd.Series(chunks_list, name="chunk_size"))
chunks_df.head()

,chunk_size
0,20800.0
1,40800.0
2,40800.0
3,21900.0
4,18500.0


In [10]:
chunks_df["prev_size"] = chunks_df.chunk_size.shift(1)
chunks_df["eq_to_prev"] = chunks_df["chunk_size"] == chunks_df["prev_size"]
chunks_df.head(3)

,chunk_size,prev_size,eq_to_prev
0,39500.0,NaN,False
1,36300.0,39500.0,False
2,39900.0,36300.0,False


In [14]:
rand_rate1 = drop_amt1.chunks["rand_rate"]

# Берем лимиты под генерацию массива чанков, в зависимости от
# полученной дропом суммы
small = drop_amt1.chunks["rcvd_small"]
medium = drop_amt1.chunks["rcvd_medium"] 
large = drop_amt1.chunks["rcvd_large"]

if drop_amt1.balance <= small["limit"]:
    low = small["min"]
    high = min(drop_amt1.balance, small["max"])

elif drop_amt1.balance <= medium["limit"]:
    low = medium["min"]
    high = min(drop_amt1.balance, medium["max"])

else:
    low = large["min"]
    high = min(drop_amt1.balance, large["max"])
    
step = drop_amt1.chunks["step"]
chunks_num = np.arange(start=low, stop=high + step, step=step).shape[0]
one_chunk_p = 1 / chunks_num
print(f"""balance: {drop_amt1.balance}
rand_rate: {rand_rate1}
chunks number: {chunks_num}
prob of taking one chunk: {one_chunk_p:.3f}
{rand_rate1 - (rand_rate1 * one_chunk_p):.3f} chunks won't be the same as a previous chunk size""")

balance: 10000
rand_rate: 0.9
chunks number: 8
prob of taking one chunk: 0.125
0.787 chunks won't be the same as a previous chunk size


In [11]:
chunks_df.eq_to_prev.value_counts(normalize=True)

eq_to_prev
False    0.908
True     0.092
Name: proportion, dtype: float64

In [12]:
chunks_df.chunk_size.agg(["min","max"])

min    18000.0
max    41900.0
Name: chunk_size, dtype: float64

**Тест `reduce_amt`**

In [13]:
drop_amt1.balance = 10000
drop_amt1.first_decl = 10000
drop_amt1.last_amt = 7500
print("manual: ", drop_amt1.first_decl * drop_amt1.reduce_share)
drop_amt1.reduce_amt(online=True)

manual:  2500.0


5000.0

**тест `one_operation`**

In [ ]:
# chunks: # Настройки создания суммы, для распределения полученных денег по частям
# atm_min: 10000 # Минимальная сумма для снятий в банкомате.
# atm_share: 0.5 # Доля от баланса, которую дроп снимает в случае снятия в банкомате
# step: 1000 # Шаг возможных сумм. Чем меньше шаг, тем больше вариантов.
# rand_rate: 0.9 # В скольких случаях размер суммы должен быть семплирован, а не взят размер предыдущей транзакции
# rcvd_small: # Категория полученной дропом суммы
#   limit: 10000 # Категория все что "до" включительно
#   min: 3000 # # Минимальная сумма чанка для этой категории
#   max: 10000 # Максимальная сумма чанка для этой категории
# rcvd_medium:
#   limit: 30000
#   min: 5000
#   max: 25000
# rcvd_large: # порог не прописываем, т.к. это все что выше medium
#   min: 10000
#   max: 40000

In [14]:
small = drop_amt1.chunks["rcvd_small"]
medium = drop_amt1.chunks["rcvd_medium"] 
large = drop_amt1.chunks["rcvd_large"]

**Ниже ячейка для кейсов:**
- Не отклонена. Онлайн. Частями. Small
- Не отклонена. Онлайн. Частями. Medium
- Не отклонена. Онлайн. Частями. Large

- `min_lim` всегда стоит `min_lim = small["min"]` т.к. баланс уменьшается и могут срабатывать условия для `small`, `medium`
- `drop_amt1.balance` для large ставить как `medium["limit"] + число` т.к. у large нет своего лимита, используется все что выше `medium["limit"]`

In [18]:
i = 0
min_lim = small["min"]
max_lim = large["max"]

while i < 1000:
    drop_amt1.reset_cache()
    drop_amt1.balance = medium["max"] + 30000

    all_ops = []
    
    while drop_amt1.balance > 0:
        one_op1 = drop_amt1.one_operation(online=True, declined=False, in_chunks=True)
        all_ops.append(one_op1)
        i += 1
    all_ops_ser = pd.Series(all_ops)
    # для минимального берем серию без последнего элемента. Т.к. там обычно уже остаток, а не чанк
    if all_ops_ser.shape[0] == 1:
        assert all_ops_ser.min() >= min_lim, \
                            f"Min is lower than min limit. {all_ops_ser.min()}"
    else:
        assert all_ops_ser.iloc[0:(all_ops_ser.shape[0] - 1)].min() >= min_lim, \
                            f"Min is lower than min limit. {all_ops_ser.iloc[0:(all_ops_ser.shape[0] - 1)].min()}"
        
    assert all_ops_ser.max() <= max_lim, "Max is higher than min limit."

In [19]:
all_ops_ser

0    14000
1    12000
2    21000
3     6000
4     2000
dtype: int64

In [20]:
atm_min = drop_amt1.chunks["atm_min"]

In [21]:
drop_amt1.reset_cache()
drop_amt1.balance = atm_min
print(drop_amt1.balance)
all_ops2 = []

while drop_amt1.balance > 0:
    one_op2 = drop_amt1.one_operation(online=False, declined=False, in_chunks=True)
    all_ops2.append(one_op2)
all_ops_ser = pd.Series(all_ops2)
all_ops_ser

10000


0    10000
dtype: int64

**Тест отклоненных транзакций. Онлайн/оффлайн**

In [22]:
drop_amt1.chunks["atm_min"]

10000

In [23]:
drop_amt1.reset_cache(life_end=True)
drop_amt1.balance = atm_min + 37000
print(drop_amt1.balance)
one_op_list1 = []
while True:
    # print(drop_amt1.first_decl)
    one_op1 = drop_amt1.one_operation(online=False, declined=True, in_chunks=True)
    
    one_op_list1.append(one_op1)
    if len(one_op_list1) == 10:
        break
print(drop_amt1.balance, "\n", one_op_list1)

47000
47000 
 [26900.0, 20200.0, 13500.0, 10000, 10000, 10000, 10000, 10000, 10000, 10000]


In [24]:
cum_sub_df = pd.DataFrame()
cum_sub_df["amt"] = np.arange(0, 11, step=1)
cum_sub_df["amt"] = - (26900 * 0.25 // 100 * 100)
cum_sub_df.loc[0, "amt"] = 26900
cum_sub_df.amt.cumsum()

0     26900.0
1     20200.0
2     13500.0
3      6800.0
4       100.0
5     -6600.0
6    -13300.0
7    -20000.0
8    -26700.0
9    -33400.0
10   -40100.0
Name: amt, dtype: float64

## Класс `DropTimeHandler`
- управление временем транзакций дропа

In [238]:
# <------------------------- ГОТОВЫЙ КЛАСС ВСТАВИТЬ--------------------------

**Тест `DropTimeHandler`**

In [1]:
# Временный импорт
import os
import yaml
import pandas as pd
import numpy as np
os.chdir("..")

# Общие настройки
with open("./config/base.yaml") as f:
    base_cfg = yaml.safe_load(f)
# Настройки фрода
with open("./config/fraud.yaml") as f:
    fraud_cfg = yaml.safe_load(f)
# Настройки фрода для дропов
with open("./config/drops.yaml", encoding="utf8") as f:
    drops_cfg = yaml.safe_load(f)
# Настройки времени
with open("./config/time.yaml") as f:
    time_cfg = yaml.safe_load(f)

In [2]:
# Временный reload для разработки
# import data_generator.fraud.drops.base
# import data_generator.indev
# import data_generator.configs

# importlib.reload(data_generator.indev)
# importlib.reload(data_generator.configs)
# importlib.reload(data_generator.fraud.drops.base)


# from data_generator.configs import DropDistributorCfg
from data_generator.indev import DropConfigBuilder
from data_generator.fraud.drops.time import DropTimeHandler
from data_generator.general_time import *

drop_cfg_build = DropConfigBuilder(base_cfg=base_cfg, fraud_cfg=fraud_cfg, drop_cfg=drops_cfg)
# config2 = drop_cfg_build.build_dist_cfg()
config2 = drop_cfg_build.build_purch_cfg()
drop_time_hanler1 = DropTimeHandler(configs=config2)

In [3]:
two_way_deltas1 = pd.Series([drop_time_hanler1.get_time_delta(two_way=True) for _ in range(1000)])
two_way_deltas1.mean(), two_way_deltas1.min(), two_way_deltas1.max()

(np.float64(-7.073), np.int64(-180), np.int64(180))

In [4]:
pos_deltas1 = pd.Series([drop_time_hanler1.get_time_delta(two_way=False) for _ in range(1000)])
pos_deltas1.mean(), pos_deltas1.min(), pos_deltas1.max()

(np.float64(106.413), np.int64(30), np.int64(180))

In [5]:
for _ in range(3):
    drop_time_hanler1.txns_count(receive=True, reset=False)
    drop_time_hanler1.txns_count(receive=False, reset=False)

In [6]:
drop_time_hanler1.in_txns, drop_time_hanler1.out_txns

(3, 3)

In [7]:
drop_time_hanler1.txns_count(receive=True, reset=True)
drop_time_hanler1.in_txns, drop_time_hanler1.out_txns

(1, 0)

In [8]:
drop_time_hanler1.in_txns = 2
drop_time_hanler1.out_txns = 4
drop_time_hanler1.txns_count(receive=True, reset=True)
drop_time_hanler1.in_txns, drop_time_hanler1.out_txns

(1, 0)

**`DropTimeHandler.get_txn_time` тест**  
Тест-кейсы
- Если это самая первая транзакция. Т.к. активность дропа начинается с входящей транзакции
- Не первая транзакция и достигнут лимит входящих транзакций для периода активности
- Не первая транзакция и достигнут лимит исходящих транзакций для периода активности
- Не первая транзакция. Лимиты не достигнуты. Транзакция входящая
- Не первая транзакция. Лимиты не достигнуты. Транзакция исходящая 

In [3]:
start_time = pd.to_datetime("2025-07-02 11:15:00", format="%Y-%m-%d %H:%M:%S")
start_unix = pd_timestamp_to_unix(start_time)
start_time, start_unix

(Timestamp('2025-07-02 11:15:00'), 1751454900)

In [4]:
last_time = pd.to_datetime("2025-07-02 18:31:00", format="%Y-%m-%d %H:%M:%S")
last_unix = pd_timestamp_to_unix(last_time)
last_time, last_unix

(Timestamp('2025-07-02 18:31:00'), 1751481060)

1. Если это самая первая транзакция. Т.к. активность дропа начинается с входящей транзакции

In [5]:
drop_time_hanler1.reset_cache()
drop_time_hanler1.in_txns = 0
drop_time_hanler1.out_txns = 0
droptime1, dropunix1 = drop_time_hanler1.get_txn_time(receive=True, in_txns=0)
droptime1, dropunix1

(Timestamp('2025-01-10 06:37:00'), np.int64(1736491020))

In [6]:
drop_time_hanler1.last_unix, drop_time_hanler1.in_txns, drop_time_hanler1.out_txns

(np.int64(1736491020), 1, 0)

2. Не первая транзакция и достигнут лимит входящих транзакций для периода активности
- текущая входящая - receive=True
- текущая исходящая - receive=False

In [7]:
start_time, last_time

(Timestamp('2025-07-02 11:15:00'), Timestamp('2025-07-02 18:31:00'))

In [8]:
drop_time_hanler1.reset_cache()
drop_time_hanler1.in_txns = drop_time_hanler1.in_lim
drop_time_hanler1.out_txns = drop_time_hanler1.out_lim - 1
drop_time_hanler1.start_unix = start_unix
drop_time_hanler1.last_unix = last_unix
droptime2, dropunix2 = drop_time_hanler1.get_txn_time(receive=True, in_txns=drop_time_hanler1.in_lim * 2)
droptime_start2 = pd.to_datetime(drop_time_hanler1.start_unix, unit="s")
droptime2, dropunix2, droptime_start2

(Timestamp('2025-07-03 12:32:00'),
 1751545920,
 Timestamp('2025-07-03 12:32:00'))

In [9]:
drop_time_hanler1.in_txns, drop_time_hanler1.out_txns, drop_time_hanler1.start_unix, drop_time_hanler1.last_unix

(1, 0, 1751545920, 1751545920)

In [10]:
droptime2 - start_time

Timedelta('1 days 01:17:00')

3. Не первая транзакция и достигнут лимит исходящих транзакций для периода активности  
- текущая входящая - receive=True
- текущая исходящая - receive=False

In [11]:
drop_time_hanler1.reset_cache()
drop_time_hanler1.in_txns = drop_time_hanler1.in_lim - 1
drop_time_hanler1.out_txns = drop_time_hanler1.out_lim
drop_time_hanler1.start_unix = start_unix
drop_time_hanler1.last_unix = last_unix
droptime3, dropunix3 = drop_time_hanler1.get_txn_time(receive=False, in_txns=drop_time_hanler1.in_lim * 2)
droptime_start3 = pd.to_datetime(drop_time_hanler1.start_unix, unit="s")
droptime3, dropunix3, droptime_start3

(Timestamp('2025-07-03 09:13:00'),
 1751533980,
 Timestamp('2025-07-03 09:13:00'))

In [12]:
drop_time_hanler1.in_txns, drop_time_hanler1.out_txns, drop_time_hanler1.start_unix, drop_time_hanler1.last_unix

(0, 1, 1751533980, 1751533980)

In [13]:
droptime3 - start_time

Timedelta('0 days 21:58:00')

4. Не первая транзакция. Лимиты не достигнуты. Транзакция **входящая**

In [14]:
drop_time_hanler1.reset_cache()
drop_time_hanler1.in_txns = drop_time_hanler1.in_lim - 1
drop_time_hanler1.out_txns = drop_time_hanler1.out_lim - 1
print(f"In txns: {drop_time_hanler1.in_txns} \nOut txns: {drop_time_hanler1.out_txns}")
drop_time_hanler1.start_unix = start_unix
drop_time_hanler1.last_unix = last_unix
droptime4, dropunix4 = drop_time_hanler1.get_txn_time(receive=True, in_txns=drop_time_hanler1.in_lim * 2)
droptime_start4 = pd.to_datetime(drop_time_hanler1.start_unix, unit="s")
droptime4, dropunix4, droptime_start4

In txns: 1 
Out txns: 4


(Timestamp('2025-07-02 20:34:00'),
 1751488440,
 Timestamp('2025-07-02 11:15:00'))

In [15]:
drop_time_hanler1.in_txns, drop_time_hanler1.out_txns, drop_time_hanler1.start_unix, drop_time_hanler1.last_unix

(2, 4, 1751454900, 1751488440)

In [16]:
droptime4 - last_time

Timedelta('0 days 02:03:00')

5. Не первая транзакция. Лимиты не достигнуты. Транзакция **исходящая**

In [17]:
drop_time_hanler1.reset_cache()
drop_time_hanler1.in_txns = drop_time_hanler1.in_lim - 1
drop_time_hanler1.out_txns = drop_time_hanler1.out_lim - 1
print(f"In txns: {drop_time_hanler1.in_txns} \nOut txns: {drop_time_hanler1.out_txns}")
drop_time_hanler1.start_unix = start_unix
drop_time_hanler1.last_unix = last_unix
droptime5, dropunix5 = drop_time_hanler1.get_txn_time(receive=False, in_txns=drop_time_hanler1.in_lim * 2)
droptime_start5 = pd.to_datetime(drop_time_hanler1.start_unix, unit="s")
droptime5, dropunix5, droptime_start5

In txns: 1 
Out txns: 4


(Timestamp('2025-07-02 19:59:00'),
 1751486340,
 Timestamp('2025-07-02 11:15:00'))

In [18]:
drop_time_hanler1.in_txns, drop_time_hanler1.out_txns, drop_time_hanler1.start_unix, drop_time_hanler1.last_unix

(1, 5, 1751454900, 1751486340)

In [19]:
droptime5 - last_time

Timedelta('0 days 01:28:00')

## Класс `DistBehaviorHandler`  
Управление поведением дропов распределителей

In [ ]:
# ---------------------------- ВСТАВИТЬ КЛАСС ----------------------------------

**Тест DistBehaviorHandler**

In [1]:
# Временный импорт
import os
import yaml
import pandas as pd
import numpy as np
os.chdir("..")

# Общие настройки
with open("./config/base.yaml") as f:
    base_cfg = yaml.safe_load(f)
# Настройки фрода
with open("./config/fraud.yaml") as f:
    fraud_cfg = yaml.safe_load(f)
# Настройки фрода для дропов
with open("./config/drops.yaml", encoding="utf8") as f:
    drops_cfg = yaml.safe_load(f)
# Настройки времени
with open("./config/time.yaml") as f:
    time_cfg = yaml.safe_load(f)

In [3]:
# import data_generator.fraud.drops.behavior
# import data_generator.indev
# import data_generator.configs

# importlib.reload(data_generator.indev)
# importlib.reload(data_generator.configs)
# importlib.reload(data_generator.fraud.drops.behavior)

from data_generator.indev import DropConfigBuilder
# from data_generator.configs import DropDistributorCfg
from data_generator.fraud.drops.behavior import DistBehaviorHandler
from data_generator.fraud.drops.base import DropAmountHandler

drop_cfg_build = DropConfigBuilder(base_cfg=base_cfg, fraud_cfg=fraud_cfg, drop_cfg=drops_cfg)
config3 = drop_cfg_build.build_dist_cfg()
drop_amt1 = DropAmountHandler(configs=config3)
dist_behav1 = DistBehaviorHandler(configs=config3, amt_hand=drop_amt1)

**`sample_scenario`**

`split_rate` = 0.7  
Тест `large_balance`: баланс больше чем `DistBehaviorHandler.trf_max`

In [1242]:
# Вероятности сценариев для large_balance
split_rate = 0.7
p_atm_and_trf = split_rate * 0.5
p_split_trf = split_rate * 0.5
p_atm = 1 - split_rate
print(f"""atm+transfer prob: {p_atm_and_trf}
split_transfer prob: {p_split_trf}
atm prob: {p_atm:.3f}""")
assert sum([p_atm_and_trf, p_split_trf, p_atm]) == 1

atm+transfer prob: 0.35
split_transfer prob: 0.35
atm prob: 0.300


In [4]:
all_scens1 = []
i = 0
while i < 2000:
    drop_amt1.reset_cache()
    drop_amt1.balance = dist_behav1.trf_max + 1000
    assert drop_amt1.balance > dist_behav1.trf_max, "Balance is below trf_max"
    dist_behav1.sample_scenario()
    # print(dist_behav1.scen)
    all_scens1.append(dist_behav1.scen)
    i += 1
all_scens_ser1 = pd.Series(all_scens1)

In [5]:
all_scens_ser1.value_counts(normalize=True)

split_transfer    0.3775
atm+transfer      0.3270
atm               0.2955
Name: proportion, dtype: float64

`split_rate` = 0.7  
Тест `atm_eligible`: баланс больше чем `DistBehaviorHandler.atm_min`, но меньше чем `DistBehaviorHandler.trf_max`

In [1245]:
# Вероятности сценариев для atm_eligible
split_rate = 0.7
p_atm_and_trf = split_rate * 0.5
p_split_trf = split_rate * 0.5
p_trf = (1 - split_rate) * 0.5
p_atm = (1 - split_rate) * 0.5
print(f"""atm+transfer prob: {p_atm_and_trf}
split_transfer prob: {p_split_trf}
transfer prob: {round(p_trf, 3)}
atm prob: {round(p_atm, 3)}""")
assert sum([p_atm_and_trf, p_split_trf, p_trf, p_atm]) == 1

atm+transfer prob: 0.35
split_transfer prob: 0.35
transfer prob: 0.15
atm prob: 0.15


In [6]:
all_scens2 = []
i = 0
while i < 2000:
    drop_amt1.reset_cache()
    drop_amt1.balance = dist_behav1.atm_min + 1000
    assert drop_amt1.balance < dist_behav1.trf_max, "Balance exceeds trf_max"
    dist_behav1.sample_scenario()
    all_scens2.append(dist_behav1.scen)
    i += 1
all_scens_ser2 = pd.Series(all_scens2)

In [7]:
all_scens_ser2.value_counts(normalize=True)

atm+transfer      0.3455
split_transfer    0.3420
atm               0.1610
transfer          0.1515
Name: proportion, dtype: float64

`split_rate` = 0.7  
Тест `split_eligible`: баланс больше чем `DistBehaviorHandler.trf_min * 2`,  
но меньше чем `DistBehaviorHandler.atm_min` и `DistBehaviorHandler.trf_max`

In [8]:
# Вероятности сценариев для split_eligible
split_rate = 0.7
p_split_trf = split_rate
p_trf = 1 - split_rate
print(f"""split_transfer prob: {p_split_trf}
transfer prob: {round(p_trf, 3)}""")
assert sum([p_split_trf, p_trf]) == 1

split_transfer prob: 0.7
transfer prob: 0.3


In [9]:
all_scens3 = []
i = 0
while i < 3000:
    drop_amt1.reset_cache()
    drop_amt1.balance = dist_behav1.trf_min * 2
    assert drop_amt1.balance < dist_behav1.atm_min, "Balance exceeds atm_min"
    assert drop_amt1.balance < dist_behav1.trf_max, "Balance exceeds trf_max"
    dist_behav1.sample_scenario()
    all_scens3.append(dist_behav1.scen)
    i += 1
all_scens_ser3 = pd.Series(all_scens3)

In [10]:
all_scens_ser3.value_counts(normalize=True)

split_transfer    0.692333
transfer          0.307667
Name: proportion, dtype: float64

`split_rate` = 0.7  
Остальные случаи. баланс меньше чем `DistBehaviorHandler.trf_min * 2`

In [1251]:
all_scens4 = []
i = 0
while i < 3000:
    drop_amt1.reset_cache()
    drop_amt1.balance = dist_behav1.trf_min + 1000
    assert drop_amt1.balance < dist_behav1.trf_min * 2, "Balance exceeds the limit"
    dist_behav1.sample_scenario()
    all_scens4.append(dist_behav1.scen)
    i += 1
all_scens_ser4 = pd.Series(all_scens4)

In [1252]:
all_scens_ser4.value_counts(normalize=True)

transfer    1.0
Name: proportion, dtype: float64

**`in_chunks_val`**

In [1253]:
# Список сценариев
# "transfer"
# "atm"
# "atm+transfer"
# "split_transfer"

dist_behav1.scen = "split_transfer"
dist_behav1.in_chunks_val()
dist_behav1.in_chunks

True

**`guide_scenario`**

In [1256]:
# Список сценариев
# "atm+transfer"
# "atm"
# "split_transfer"
# "transfer"

dist_behav1.scen = "transfer"
dist_behav1.guide_scenario()
dist_behav1.online

True

**`to_drop`**

In [1257]:
print(dist_behav1.to_drop_rate)
to_drop_list1 = []
for _ in range(100):
    to_drop_list1.append(dist_behav1.to_drop)
to_drop_ser1 = pd.Series(to_drop_list1)
to_drop_ser1.value_counts(normalize=True)

0.1


False    0.92
True     0.08
Name: proportion, dtype: float64

**`to_crypto`**

In [1258]:
dist_behav1.online = True
print(dist_behav1.crypto_rate)
to_drop_list2 = []

for _ in range(100):
    to_drop_list2.append(dist_behav1.to_drop)
to_drop_ser2 = pd.Series(to_drop_list2)
to_drop_ser2.value_counts(normalize=True)

0.1


False    0.91
True     0.09
Name: proportion, dtype: float64

**`stop_after_decline`**

In [424]:
dist_behav1.attempts = 2
dist_behav1.stop_after_decline(declined=True)

False

**`reset_cache`**

In [1261]:
dist_behav1.attempts = 3
dist_behav1.scen = "atm"
dist_behav1.online = True
dist_behav1.in_chunks = False
dist_behav1.reset_cache(all=True)
dist_behav1.attempts, dist_behav1.scen, dist_behav1.online, dist_behav1.in_chunks

(0, None, None, None)

**`attempts_after_decline`**  
self.online = True, self.online = False

In [1264]:
att_vals1 = []
dist_behav1.online = True

for _ in range(100):
    dist_behav1.attempts_after_decline(declined=True)
    att_vals1.append(dist_behav1.attempts)
att_vals_ser1 = pd.Series(att_vals1)

In [1265]:
att_vals_ser1.agg(["min","mean", "max"])

min     0.00
mean    1.94
max     4.00
dtype: float64

**`deduct_attempts`**

In [434]:
dist_behav1.attempts = 1
dist_behav1.deduct_attempts(declined=True, receive=True)
dist_behav1.attempts

1

**Расчет вероятностей сценариев в `DistBehaviorHandler`**

In [ ]:
# inbound_amt: 
#   low: 5000
#   high: 100000
#   mean: 30000
#   std: 20000

In [308]:
# Баланс генерируется из обрезанного нормального распределения
in_low1 = 5000
in_high1 = 100000
mean1 = 30000
std1 = 20000
a1 = (in_low1 - mean1) / std1
b1 = (in_high1 - mean1) / std1

split_rate = 0.7
# Есть лимиты для сумм операций
atm_min1 = 10000 # минимальная сумма для снятия. Частями или целиком определяется split_rate
trf_lim = 40000 # максимальная сумма для перевода. Если больше - дробим на части
trf_min = 3000*2 # минимальная сумма для перевода по частям. Если меньше - переводим целиком

In [309]:
# вероятность что сумма будет больше trf_lim
p_trf_lim = round(truncnorm(a=a1, b=b1, loc=mean1, scale=std1).sf(x=trf_lim), 3)
p_trf_lim

np.float64(0.345)

In [310]:
# Вероятность что сумма больше atm_min1 но меньше trf_lim
p_atm_min = round(truncnorm(a=a1, b=b1, loc=mean1, scale=std1).sf(x=atm_min1) - p_trf_lim, 3)
p_atm_min

np.float64(0.596)

In [311]:
# вероятность что сумма больше trf_min, но меньше trf_lim и atm_min1
p_trf_min = round(truncnorm(a=a1, b=b1, loc=mean1, scale=std1).sf(x=trf_min) - p_trf_lim - p_atm_min, 3)
p_trf_min

np.float64(0.048)

In [312]:
split_rate = 0.7 # доля когда будем переводить частями

(p_trf_lim + p_atm_min + p_trf_min) * split_rate

np.float64(0.6922999999999999)

In [313]:
round(truncnorm(a=a1, b=b1, loc=mean1, scale=std1).sf(x=trf_min), 3) * split_rate

np.float64(0.6922999999999999)

In [315]:

# conditions = [
#             (balance > self.trf_lim, ["atm","split_transfer", "atm+transfer"]),
#             (balance >= self.atm_min, ["transfer", "atm", "split_transfer", "atm+transfer"]),
#             (balance >= self.trf_min * 2, ["transfer","split_transfer"]),
#             "transfer",
#         ]
# Вероятность случаев перевода частями если по условиям выше (синтаксически неправильно) и без split_rate
p_trf_lim * 2/3 + p_atm_min * 1/2 + p_trf_min * 1/2

np.float64(0.552)

In [645]:

# conditions = [
#             (balance > self.trf_lim, ["atm","split_transfer", "atm+transfer"]),
#             (balance >= self.atm_min, ["transfer", "atm", "split_transfer", "atm+transfer"]),
#             (balance >= self.trf_min * 2, ["transfer","split_transfer"]),
#             "transfer",
#         ]
# Вероятность случаев перевода в крипту по условиям выше (синтаксически неправильно)
crypto_rate = 0.14
(p_trf_lim * 2/3 + p_atm_min * 3/4 + p_trf_min) * crypto_rate

np.float64(0.1015)

## Класс `PurchBehaviorHandler`  
- Управление поведением дропа покупателя

In [ ]:
# -------------------------- ВСТАВИТЬ ГОТОВЫЙ КЛАСС --------------------------

**Тест `PurchBehaviorHandler`**

In [1]:
# Временный импорт
import os
import yaml
import pandas as pd
import numpy as np
os.chdir("..")

# Общие настройки
with open("./config/base.yaml") as f:
    base_cfg = yaml.safe_load(f)
# Настройки фрода
with open("./config/fraud.yaml") as f:
    fraud_cfg = yaml.safe_load(f)
# Настройки фрода для дропов
with open("./config/drops.yaml", encoding="utf8") as f:
    drops_cfg = yaml.safe_load(f)
# Настройки времени
with open("./config/time.yaml") as f:
    time_cfg = yaml.safe_load(f)

In [2]:
from data_generator.indev import DropConfigBuilder
from data_generator.fraud.drops.behavior import PurchBehaviorHandler
from data_generator.fraud.drops.base import DropAmountHandler

drop_cfg_build = DropConfigBuilder(base_cfg=base_cfg, fraud_cfg=fraud_cfg, drop_cfg=drops_cfg)
configs = drop_cfg_build.build_purch_cfg()
drop_amt1 = DropAmountHandler(configs=configs)
purch_behav1 = PurchBehaviorHandler(configs=configs, amt_hand=drop_amt1)

`split_rate` = 0.7  
Тест `large_balance`: баланс больше чем amt_max

In [3]:
all_scens1 = []
i = 0
while i < 2000:
    drop_amt1.reset_cache()
    drop_amt1.balance = purch_behav1.amt_max + 1000
    assert drop_amt1.balance > purch_behav1.amt_max, "Balance is below amt_max"
    purch_behav1.sample_scenario()
    # print(purch_behav1.scen)
    all_scens1.append(purch_behav1.scen)
    i += 1
all_scens_ser1 = pd.Series(all_scens1)

all_scens_ser1.value_counts(normalize=True)

split_money    1.0
Name: proportion, dtype: float64

`split_rate` = 0.7  
Тест `split_eligible`: баланс больше чем `self.amt_min * 2`,  
но меньше чем `self.amt_max`

In [5]:
# Вероятности сценариев для split_eligible
split_rate = 0.7
p_split = split_rate
p_one = 1 - split_rate
print(f"""split_money prob: {p_split}
one_purchase prob: {round(p_one, 3)}""")
assert sum([p_split, p_one]) == 1

split_money prob: 0.7
one_purchase prob: 0.3


In [6]:
all_scens3 = []
i = 0
while i < 3000:
    drop_amt1.reset_cache()
    drop_amt1.balance = purch_behav1.amt_min * 2
    assert drop_amt1.balance < purch_behav1.amt_max, "Balance exceeds amt_max"
    purch_behav1.sample_scenario()
    all_scens3.append(purch_behav1.scen)
    i += 1
all_scens_ser3 = pd.Series(all_scens3)

all_scens_ser3.value_counts(normalize=True)

split_money     0.697333
one_purchase    0.302667
Name: proportion, dtype: float64

`split_rate` = 0.7  
баланс меньше чем `amt_min * 2`

In [8]:
all_scens4 = []
i = 0
while i < 3000:
    drop_amt1.reset_cache()
    drop_amt1.balance = purch_behav1.amt_min + 1000
    assert drop_amt1.balance < purch_behav1.amt_min * 2, "Balance exceeds the limit"
    purch_behav1.sample_scenario()
    all_scens4.append(purch_behav1.scen)
    i += 1
all_scens_ser4 = pd.Series(all_scens4)

all_scens_ser4.value_counts(normalize=True)

one_purchase    1.0
Name: proportion, dtype: float64

# **`in_chunks_val`**

In [4]:
# Список сценариев
# "split_money"
# "one_purchase"

purch_behav1.scen = "one_purchase"
purch_behav1.in_chunks_val()
purch_behav1.in_chunks

False

**`stop_after_decline`**

In [8]:
purch_behav1.attempts = 0
purch_behav1.stop_after_decline(declined=True)

True

**`reset_cache`**

In [9]:
purch_behav1.attempts = 3
purch_behav1.scen = "split_money"
purch_behav1.online = True
purch_behav1.in_chunks = False
purch_behav1.reset_cache(all=True)
purch_behav1.attempts, purch_behav1.scen, purch_behav1.online, purch_behav1.in_chunks

(0, None, True, None)

**`attempts_after_decline`**  
declined = True/False

In [3]:
att_vals1 = []

for _ in range(100):
    purch_behav1.attempts_after_decline(declined=True)
    att_vals1.append(purch_behav1.attempts)
att_vals_ser1 = pd.Series(att_vals1)

att_vals_ser1.agg(["min","mean", "max"])

min     0.00
mean    1.87
max     4.00
dtype: float64

**`deduct_attempts`**

In [6]:
purch_behav1.attempts = 4
purch_behav1.deduct_attempts(declined=True, receive=False)
purch_behav1.attempts

3

## Конец информативной части

# Старый код, функции и т.д.